# 0. Prerequisites

This notebook is made to run on google colab.

The first cells allow to set up the environment and to place all data at the right place. 

In [ ]:
# Mount drive to have access to some data.

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Clone the git repository containing the code and installing correctly the dependencies

!git clone https://github.com/tanguy-magne/DirectVoxGo-NPM3D.git
%cd DirectVoxGo-NPM3D/DirectVoxGO

!pip install -r requirements.txt
!pip install ninja

import torch
assert(torch.__version__ == '1.10.0+cu111')
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html

!pip install einops

!ls

# 1. Dataset

This part of the code is made to set all dataset in the working environment.

In [3]:
PATH_DATA = "/content/DirectVoxGo-NPM3D/DirectVoxGO/data" 
!mkdir -p {PATH_DATA}

In [4]:
# This dataset have to be dowloaded from internet, at this link 
# https://drive.google.com/drive/folders/128yBriW1IG_3NJ5Rp7APSTZsJqdJdfc1
# The two .zip files that have to be put on your drive are nerf_synthetic.zip and nerf_llff_data.zip
# Then the variable PATH_TO_DATA_DRIVE has to be updated
# Note that you can skip this part by commenting it and just not use this dataset

PATH_TO_DATA_DRIVE = "/content/drive/MyDrive/MVA/2.3_npm3d/Project"
datasets = {
  "nerf_synthetic" : f"{PATH_TO_DATA_DRIVE}/nerf_synthetic.zip",
  "nerf_llff_data" : f"{PATH_TO_DATA_DRIVE}/nerf_llff_data.zip"
}

for dataset, dataset_path in datasets.items():
  !unzip -q {dataset_path} -d {PATH_DATA}

unwanted_MACOSX = f"{PATH_DATA}/__MACOSX"
!rm -r {unwanted_MACOSX}

In [5]:
# The deepvoxels dataset requires the same procedure. The link to dowload it is the following
# https://drive.google.com/drive/folders/1ScsRlnzy9Bd_n-xw83SP-0t548v63mPH
# You need to download the .zip file synthetic_scenes.zip and place it with the other on your drive

PATH_DEEPVOXELS = PATH_DATA + "/deepvoxels"
PATH_DEEPVOXELS_ON_DRIVE = f"{PATH_TO_DATA_DRIVE}/synthetic_scenes.zip"
!unzip -q {"/content/drive/MyDrive/MVA/2.3_npm3d/Project/synthetic_scenes.zip"} -d {PATH_DEEPVOXELS}

In [6]:
# These datasets are available online and don't require anything to work, you just need to run this code

datasets  = {
  "Synthetic_NSVF" : "https://dl.fbaipublicfiles.com/nsvf/dataset/Synthetic_NSVF.zip",
  "BlendedMVS": "https://dl.fbaipublicfiles.com/nsvf/dataset/BlendedMVS.zip",
  "TanksAndTemple": "https://dl.fbaipublicfiles.com/nsvf/dataset/TanksAndTemple.zip",
}

for dataset, url in datasets.items():
  dataset_path = f"{PATH_DATA}/{dataset}.zip"
  !wget -q -O {dataset_path} {url}
  !unzip -q {dataset_path} -d {PATH_DATA}
  !rm {dataset_path}

In [7]:
# Custom dataset : These dataset can be found at this link : https://drive.google.com/drive/folders/1w46Rlg1ZoKXEiu3D6wn-GQT6c0Jcu1Mu?usp=sharing
# They should be downloaded and be place on your drive alonside with the NeRF synthetic dataset

PATH_NOUNOURS = f"{PATH_DATA}/nerf_llff_data"
PATH_NOUNOURS_DRIVE = f"{PATH_TO_DATA_DRIVE}/nounours.zip"
!unzip -q {PATH_NOUNOURS_DRIVE} -d {PATH_NOUNOURS}
PATH_PANDA = f"{PATH_DATA}/nerf_llff_data"
PATH_PANDA_DRIVE = f"{PATH_TO_DATA_DRIVE}/panda.zip"
!unzip -q {PATH_PANDA_DRIVE} -d {PATH_PANDA}

# 3. Training and Evaluating models

To change some parameters, you can simply go to the config files corresponding to the scene you want to train a model on, and change the correponding paramaters. This is how I run all my experiments and ablations study.

Here are the paramaters I changed in my experiments, that corresponds to the main contributions of the article :
- To change parameters of the coarse stage of the model (only available if not using forward facing scene, which corresponds to LLFF dataset)
  - To change the value of $\alpha^{(init)(c)}$, the initial value of the $\alpha$ parameter in the coarse stage add this to the config `coarse_model_and_render = dict(alpha_init=value)` with value being the value you want
  - To use or not the view-count-based learning rate, add this to the config `coarse_train = dict(pervoxel_lr=True)`. You can change the value to False to not use this feature. 
- To change parameters of the fine stage of the model :
  - To change the value of $\alpha^{(init)(f)}$, the initial value of the $\alpha$ parameter in the fine stage add this to the config `fine_model_and_render = dict(alpha_init=value)` with value being the value you want
  - To use or not the view-count-based learning rate, add this to the config `fine_train = dict(pervoxel_lr=True)`. You can change the value to False to not use this feature. 
- To use pre-activation you need to go in the code :
  - If working with LLFF dataset go to `DirectVocGO/lib/dmpigo.py` file. Then comment lines 227-229 and uncomment lines 223-225
  - Otherwise go to `DirectVocGO/lib/dvgo.py` file. Then comment lines 314-316 and uncomment lines 318-320.


In [ ]:
# This line of command allow to train a model
!python run.py --config configs/llff/nounours.py --render_test --eval_ssim --eval_lpips_vgg --eval_lpips_alex

In [ ]:
# This line of command allow to evaluate a model, without running the training again 
!python run.py --config configs/llff/nounours.py --render_only --render_test --render_video --eval_ssim --eval_lpips_vgg